# COVCoNet

By Elías Jesús Ventura-Molina

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

## Carga el modelo ya entrenado con los pesos

In [2]:
model = tf.keras.models.load_model('COVID19_accuracu9629.h5')

## Carga las imagenes de entrenamiento y prueba 

In [3]:
train_dir = 'C:/Users/lept2/Documents/CIC/B20/CoreCongress/dataset_3_classes/train'
#train_dir = 'C:/Users/lept2/Documents/CIC/B20/CoreCongress/four_classes/train'
test_dir = 'C:/Users/lept2/Documents/CIC/B20/CoreCongress/dataset_3_classes/test'
#test_dir = 'C:/Users/lept2/Documents/CIC/B20/CoreCongress/four_classes/test'

In [4]:
train_datagenerator = ImageDataGenerator(rescale = 1./255, validation_split = 0.3)
test_datagenerator = ImageDataGenerator(rescale = 1./255)
BATCH_SIZE_TRAINING = 1
BATCH_SIZE_VALIDATION = 1

In [5]:
altura = 224
ancho = 224

## Conjunto de entrenamiento 

In [6]:
training_set = train_datagenerator.flow_from_directory(
    train_dir,
    target_size = (altura, ancho),
    class_mode='categorical',
    subset = 'training',
    batch_size = BATCH_SIZE_TRAINING
)
training_set.class_indices

Found 140 images belonging to 3 classes.


{'covid': 0, 'normal': 1, 'pneumonia_bac': 2}

## Conjunto de validación 

In [7]:
validation_set = train_datagenerator.flow_from_directory(
    train_dir,
    target_size = (altura, ancho),
    class_mode='categorical',
    subset = 'validation',
    batch_size = BATCH_SIZE_VALIDATION
)
training_set.class_indices

Found 60 images belonging to 3 classes.


{'covid': 0, 'normal': 1, 'pneumonia_bac': 2}

## Conjunto de prueba

In [8]:
testing_set = test_datagenerator.flow_from_directory(
    test_dir,
    target_size = (altura, ancho),
    class_mode='categorical',
    batch_size = 1,
    shuffle = False
)
testing_set.class_indices

Found 30 images belonging to 3 classes.


{'covid': 0, 'normal': 1, 'pneumonia_bac': 2}

## Diseño de la red neuronal

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (2,2), input_shape = (altura, ancho, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (2,2), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (2,2), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (2,2), activation = 'relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(150, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
])


## Compilación de la red neuronal y arquitectura 

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'rmsprop',
    metrics = ['accuracy']
)
model.summary()

## Entranamiento de la red neuronal 

In [ ]:
histoy = model.fit_generator(
    training_set,
    epochs = 10,
    validation_data = validation_set,
    verbose = 1
)

## Evaluación de la red neuronal con el conjunto de prueba 

In [9]:
resultados = model.evaluate(testing_set)

  ...
    to  
  ['...']
30/30 [==============================] - 2s 78ms/step - loss: 0.2552 - accuracy: 0.9667


## Matriz de confusión

In [10]:
Y_pred = model.predict_generator(testing_set, 1042 // 1+1)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
print(testing_set.classes)
print('Confusion Matrix')
print(confusion_matrix(testing_set.classes, y_pred))
print('\n')
print('                      Reporte de Clasificacion')
print('\n')
target_names = ['covid', 'normal', 'neumonia']
print(classification_report(testing_set.classes, y_pred, target_names=target_names))

Instructions for updating:
Please use Model.predict, which supports generators.
[0 0 0 0 0 0 0 0 0 0 1 1 1 2 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]
Confusion Matrix
[[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]


                      Reporte de Clasificacion


              precision    recall  f1-score   support

       covid       1.00      1.00      1.00        10
      normal       1.00      0.90      0.95        10
    neumonia       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

